# 加密货币指数计算演示

本演示展示如何构建和计算加密货币市值加权指数的完整流程。

## 市值加权指数原理

**核心概念**：
- **市值** = 当前价格 × 流通供应量
- **权重** = 该资产市值 / 所有成分资产市值总和
- **指数值** = 基准值 × (当前总市值 / 基准总市值)

**基本流程**：选择成分 → 计算权重 → 设定基准 → 计算指数值

## 步骤0：环境准备

导入必要的函数和设置路径。

In [1]:
# 步骤0 环境准备（极简）
import sys, pathlib
root = pathlib.Path().resolve().parents[1]
if str(root) not in sys.path:
    sys.path.append(str(root))
from src.lab.minapi import daily, top, weights, index_value, index_series, show_top


INFO:src.downloaders.daily_aggregator:数据库支持已启用


## 步骤1：获取基准日数据

展示基准日期的前N大市值代币，作为指数成分。

In [2]:
# 步骤1：基准日数据
base_date = "2023-12-01"
coin_num = 15
show_top(base_date, n=coin_num, title=f"前{coin_num}大市值 (基准日: {base_date})")

✅ 项目路径: D:\Codes\crypto_index_study
🔧 使用数据库: market.db (175.9MB)
🎉 环境就绪 (quick_env)

📊 前15大市值 (基准日: 2023-12-01)


,排名,代码,币种名称,价格($),市值(1B$)
0,1,BTC,Bitcoin,"37,711.8184",738
1,2,ETH,Ethereum,"2,051.7567",247
2,3,BNB,Binance Coin,227.6026,35
3,4,XRP,Ripple,0.6058,33
4,5,SOL,Solana,59.1800,25
5,6,ADA,Cardano,0.3756,13
6,7,DOGE,Dogecoin,0.0833,12
7,8,TRX,TRON,0.1034,9
8,9,TON,Toncoin,2.3916,8
9,10,LINK,Chainlink,14.3932,8


## 步骤2：计算权重并设定基准

计算各成分的权重，设定基准指数值，展示指数构成。

In [3]:
# 步骤2：权重与基准（封装）
# 可选：如有修改minapi.py后需重载: import importlib; importlib.reload(__import__('src.lab.minapi'))
from src.lab.minapi import prepare_index_base

components, index_info, formatted = prepare_index_base(base_date, n=coin_num, base_value=1000.0)
base_index_value = index_info['base_value']
base_total_market_cap = index_info['base_market_cap']


📊 指数构成 (基准 2023-12-01)


,排名,代码,币种名称,价格($),市值(1B$),权重(%)
0,1,BTC,Bitcoin,"37,711.8184",737.660,63.60
1,2,ETH,Ethereum,"2,051.7567",246.848,21.28
2,3,BNB,Binance Coin,227.6026,35.008,3.02
3,4,XRP,Ripple,0.6058,32.670,2.82
4,5,SOL,Solana,59.1800,25.109,2.17
5,6,ADA,Cardano,0.3756,13.142,1.13
6,7,DOGE,Dogecoin,0.0833,11.842,1.02
7,8,TRX,TRON,0.1034,9.166,0.79
8,9,TON,Toncoin,2.3916,8.263,0.71
9,10,LINK,Chainlink,14.3932,8.013,0.69


## 步骤3：计算目标日指数值

使用公式 **指数值 = 基准值 × (当前总市值 / 基准总市值)** 计算指定日期的指数值。

In [4]:
# 步骤3：目标日指数
target_date = "2024-07-01"
print(f"🎯 日期: {target_date}")
index_val = index_value(base_date, target_date, n=coin_num, base_value=base_index_value)
if index_val != index_val:  # NaN 检查
    print("⚠️ 无可计算数据")
else:
    diff = index_val - base_index_value
    pct = diff / base_index_value * 100
    print(f"指数 {index_info['name']}: 基准 {base_index_value:.2f} -> {target_date} {index_val:.2f} | 变动 {diff:+.2f} ({pct:+.2f}%)")
# 可选序列：
# series_df = index_series(base_date, ["2024-01-01","2024-03-01","2024-05-01","2024-07-01"], n=coin_num, base_value=base_index_value)
# from IPython.display import display; display(series_df)

🎯 日期: 2024-07-01
📊 开始清理数据: 510 行
📊 重新计算排名: 1-510
✅ 数据清理完成: 510 行，8 列
指数 Crypto15 市值加权指数: 基准 1000.00 -> 2024-07-01 1676.46 | 变动 +676.46 (+67.65%)
📊 开始清理数据: 510 行
📊 重新计算排名: 1-510
✅ 数据清理完成: 510 行，8 列
指数 Crypto15 市值加权指数: 基准 1000.00 -> 2024-07-01 1676.46 | 变动 +676.46 (+67.65%)


## 总结

演示完成！通过4个简单步骤构建了加密货币市值加权指数：

**流程**：环境准备 → 获取基准数据 → 计算权重基准 → 计算目标指数

**核心公式**：
```
指数值 = 基准值 × (当前总市值 / 基准总市值)
```